In [1]:
import numpy as np
import collections
import gzip
import random

from scipy import stats
from scipy.interpolate import interp1d

import matplotlib.pyplot as plt
#plt.switch_backend('agg')
import matplotlib
import sys
sys.path.append('../')
sys.path.append('/usr/users/fsimone/tejaas')
# from utils import mpl_stylesheet
# mpl_stylesheet.banskt_presentation(fontfamily = 'latex-clearsans', fontsize = 18, colors = 'banskt', dpi = 72)

In [2]:
f_vcf = "/cbscratch/franco/datasets/gtex_v8/genotypes/vcfs_0.01/GTEX_v8_2019-07-29_WGS_838Indiv_Freeze_NoMissingGT_SNPfilter_MAF0.01_chr17.vcf.gz"

In [3]:
from sklearn.decomposition import PCA

SNPINFO_FIELDS = ['chrom', 'varid', 'bp_pos', 'ref_allele', 'alt_allele', 'maf']
class SnpInfo(collections.namedtuple('_SnpInfo', SNPINFO_FIELDS)):
    __slots__ = ()

from utils.containers import GeneInfo, CisMask

SNP_COMPLEMENT = {'A':'T', 'C':'G', 'G':'C', 'T':'A'}

def filter_snps(snpinfo, dosage):
    # Predixcan style filtering of snps
    newsnps = list()
    newdosage = list()
    npoly = 0
    nambi = 0
    nunkn = 0
    nlowf = 0
    nhwep = 0
    for i, snp in enumerate(snpinfo):
        pos = snp.bp_pos
        refAllele = snp.ref_allele
        effectAllele = snp.alt_allele
        rsid = snp.varid
        maf = round(snp.maf, 3)
        # Skip non-single letter polymorphisms
        if len(refAllele) > 1 or len(effectAllele) > 1:
            npoly += 1
            continue
        # Skip ambiguous strands
        if SNP_COMPLEMENT[refAllele] == effectAllele:
            nambi += 1
            continue
        # Skip unknown RSIDs
        if rsid == '.':
            nunkn += 1
            continue
        # Skip low MAF
        if not (maf >= 0.01 and maf <=0.99):
            nlowf += 1
            continue
        newsnps.append(snp)
        newdosage.append(dosage[i])
        # newdosage.append(intdosage)
    print("Removed {:d} SNPs because of non-single letter polymorphisms".format(npoly))
    print("Removed {:d} SNPs because of ambiguous strands".format(nambi))
    print("Removed {:d} SNPs because of unknown RSIDs".format(nunkn))
    print("Removed {:d} SNPs because of low MAF < 0.01".format(nlowf))
    return newsnps, np.array(newdosage)    

def read_vcf(filename, startsnp, endsnp):
    dosage = list()
    snpinfo = list()
    linenum = 0
    with gzip.open(filename, 'r') as vcf:
        for line in vcf:
            linestrip = line.decode().strip()
            if linestrip[:2] == '##': continue
            if linestrip[:6] == '#CHROM':
                linesplit = linestrip.split("\t")
                donor_ids = linesplit[9:]
            else:
                if linenum >= startsnp and linenum < endsnp:
                    linesplit = linestrip.split("\t")
                    chrom = linesplit[0]
                    if chrom.startswith("chr"):
                        chrom = int(chrom[3:])
                    else:
                        chrom = int(linesplit[0])
                    pos   = int(linesplit[1])
                    varid = linesplit[2]
                    ref   = linesplit[3]
                    alt   = linesplit[4]

                    gtindx = linesplit[8].split(':').index("GT")
                    gt = [x.split(':')[gtindx] for x in linesplit[9:]]
                    ds = [ float(int(x[0]) + int(x[2])) if len(x) == 3 and x[0] != "." and x[2] != "." else "." for x in gt ]

                    ds_notna = [float(x) for x in ds if x != "."]
                    freq = sum(ds_notna) / 2 / len(ds_notna)
                    maf = freq
                    snpdosage = [float(x) if x != '.' else 2 * freq for x in ds]

                    this_snp = SnpInfo(chrom      = chrom,
                                       bp_pos     = pos,
                                       varid      = varid,
                                       ref_allele = ref,
                                       alt_allele = alt,
                                       maf        = maf)

                    dosage.append(snpdosage)
                    snpinfo.append(this_snp)
                linenum += 1
                
                if linenum > endsnp:
                    break
    return np.array(dosage), snpinfo, donor_ids


def normalize_expr(Y):
    newY = (Y - np.mean(Y, axis = 1).reshape(-1, 1)) / np.std(Y, axis = 1).reshape(-1, 1)
    return newY

def select_donors(vcf_donors, expr_donors):
    ''' Make sure that donors are in the same order for both expression and genotype
    '''
    common_donors = [x for x in vcf_donors if x in expr_donors]
    vcfmask = np.array([vcf_donors.index(x) for x in common_donors])
    exprmask = np.array([expr_donors.index(x) for x in common_donors])
    return vcfmask, exprmask

def select_genes(info, names):
    ''' Select genes which would be analyzed. 
        Make sure the indices are not mixed up
    '''
    allowed = [x.ensembl_id for x in info]
    common  = [x for x in names if x in allowed]
    genes = [x for x in info if x.ensembl_id in common]
    indices = [names.index(x.ensembl_id) for x in genes]
    return genes, np.array(indices)

def normalize_and_center_dosage(dosage, snpinfo):
    f = [snp.maf for snp in snpinfo]
    f = np.array(f).reshape(-1, 1)
    gtcent = dosage - np.mean(dosage, axis = 1).reshape(-1, 1)
    gtnorm = (dosage - (2 * f)) / np.sqrt(2 * f * (1 - f))
    return gtcent, gtnorm

def sample_gt(snpinfo, nsample):
    nsnps = len(snpinfo)
    dosages = np.zeros(nsnps * nsample)
    i = 0
    for snp in snpinfo:
        gtcent = sample_from_maf(nsample, snp.maf)
        dosages[i,:] = gtcent
        i += 1
    return dosages    

def sample_from_maf(nsample, maf):
    dosage = np.zeros(nsample)
    mafratios = np.array([(1 - maf)**2, 2 * maf * (1 - maf), maf**2])
    nfreq  = np.random.multinomial(nsample, mafratios, size=1)[0]
    f1 = np.repeat(0, nfreq[0])
    f2 = np.repeat(1, nfreq[1])
    f3 = np.repeat(2, nfreq[2])
    x  = np.concatenate((f1,f2,f3))
    dosage = np.random.permutation(x)
    # gtnorm = (dosage - (2 * maf2d)) / np.sqrt(2 * maf2d * (1 - maf2d))
    gtcent = dosage - np.mean(dosage)

    return gtcent

def simulate_gt(nsnps, nsample):
    mafs = np.linspace(0.1, 0.9, nsnps)
    gtcent = np.zeros((nsnps, nsample))
    snpinfo = list()
    for i in range(nsnps):
        gtcent[i,:] = sample_from_maf(nsample, mafs[i])
        this_snp = SnpInfo(chrom      = 1,
                           bp_pos     = i*100,
                           varid      = "rsid"+str(i),
                           ref_allele = "A",
                           alt_allele = "G",
                           maf        = mafs[i])
        snpinfo.append(this_snp)
    return gtcent, snpinfo

def knn_correction(expr, dosage, K=30):
    pca = PCA(n_components=min(expr.shape[0], expr.shape[1]))
    print("Original dimension: {:d} x {:d}".format(expr.shape[0], expr.shape[1]))
    pca.fit(expr) # requires N x G
    expr_pca = pca.transform(expr)
    print("Reduced dimension: {:d} x {:d}".format(expr_pca.shape[0], expr_pca.shape[1]))

    def gene_distance(a, b):
        return np.linalg.norm(a - b)

    nsample = expr.shape[0]
    distance_matrix = np.zeros((nsample, nsample))
    for i in range(nsample):
        for j in range(i+1, nsample):
            dist = gene_distance(expr_pca[i,:], expr_pca[j,:])
            distance_matrix[i, j] = dist
            distance_matrix[j, i] = dist

    kneighbor = K
    gx_knn = np.zeros_like(expr)
    gt_knn = np.zeros_like(dosage)
    neighbor_list = list()

    for i in range(nsample):
        neighbors = np.argsort(distance_matrix[i, :])[:kneighbor + 1][1:]
        gx_knn[i, :] = expr[i, :] - np.mean(expr[neighbors, :], axis = 0)
        # noisy_neighbors = np.random.choice(neighbors, size = int(2 * kneighbor / 3), replace = False)
        # noisy_neighbors = np.random.choice(neighbors, size = kneighbor, replace = True )
        noisy_neighbors = neighbors
        gt_knn[:, i] = dosage[:, i] - np.mean(dosage[:, noisy_neighbors], axis = 1)
        neighbor_list.append(neighbors)

    return gx_knn, gt_knn

def get_cismasklist(snpinfo, geneinfo, chrom, window=1e6):
    chr_genes_ix = [[] for ichrm in range(22)] 
    chr_genes = [[] for ichrm in range(22)]
    if chrom is not None:
        chr_genes_ix[chrom - 1] = np.array([i for i, g in enumerate(geneinfo) if g.chrom == chrom])
        chr_genes[chrom - 1] = [geneinfo[ix] for ix in chr_genes_ix[chrom - 1]]
    else:
        for ichrm in range(22):
            chr_genes_ix[ichrm] = np.array([i for i, g in enumerate(geneinfo) if g.chrom == ichrm + 1])
            chr_genes[ichrm] = [geneinfo[ix] for ix in chr_genes_ix[ichrm]]
    genemasks = list()
    iprev = 0
    ichrmprev = 0
    for snp in snpinfo:
        pos = snp.bp_pos
        left = pos - window
        right = pos + window
        ichrm = chrom - 1 if chrom is not None else snp.chrom - 1
        iprev_started = False
        if ichrm != ichrmprev:
            iprev = 0
            ichrmprev = ichrm
        thismask = list()
        for i, g in enumerate(chr_genes[ichrm][iprev:]):
            gstart = g.start
            gend = g.end
            if gstart >= left and gstart <= right:
                # thismask.append(iprev + i)
                thismask.append(chr_genes_ix[ichrm][iprev + i])
                if not iprev_started:
                    new_start_iloc = iprev
                    iprev_started = True
            elif gend >= left and gend <= right:
                # thismask.append(iprev + i)
                thismask.append(chr_genes_ix[ichrm][iprev + i])
                if not iprev_started:
                    new_start_iloc = iprev
                    iprev_started = True
            if gstart > right:
                break
        if len(thismask) > 0:
            #genemasks.append(chr_genes_ix[np.array(thismask)])
            #iprev = thismask[0]
            genemasks.append(np.array(thismask))
            iprev = new_start_iloc
        else:
            genemasks.append(np.array([]))
    return genemasks

def compress_cismasklist(genemasks):
    cismasks = list()
    appendmask = False
    endmask = False
    setprev = False
    snplist = list()
    for i, mask in enumerate(genemasks):
        if not setprev:
            prev_mask = mask
            setprev = True
        if np.all(np.array_equal(mask, prev_mask)):
            snplist.append(i)
        else:
            appendmask = True

        if i == len(genemasks) - 1: endmask = True # no more masks to process

        if appendmask:
            thismask = CisMask(rmv_id = prev_mask, apply2 = snplist)
            cismasks.append(thismask)
            snplist = list([i])
            prev_mask = mask
            if not endmask:
                appendmask = False

        if endmask:
            # if not appendmask:
            #     snplist.append(i)
            thismask = CisMask(rmv_id = mask, apply2 = snplist)
            cismasks.append(thismask)

    return cismasks

In [31]:
# Use real genotype
# gtfull, snpinfo, gt_donors = read_vcf(f_vcf, 9080, 9105)
gtfull, snpinfo, gt_donors = read_vcf(f_vcf, 0, 1)
snp_info, gtfull = filter_snps(snpinfo, gtfull)

# Use artificial genotype
# gtfull, snp_info = simulate_gt(10000, len(gt_donors))


Removed 0 SNPs because of non-single letter polymorphisms
Removed 0 SNPs because of ambiguous strands
Removed 0 SNPs because of unknown RSIDs
Removed 0 SNPs because of low MAF < 0.01


In [32]:
# # Simulate some expression
# ngene = 15000
# nsample = 350
# gx_rand = np.random.normal(0, 1, size = nsample * ngene).reshape((ngene, nsample)) 
# gx_donors = random.sample(gt_donors, nsample)
# print(gx_rand.shape)

# Load real expression
import pandas as pd
df = pd.read_csv("/cbscratch/franco/trans-eqtl/new_preprocess_aug2019/gtex_v8/expression/tpms/sse_tpms_qcfilter.txt.protein_coding_filtered", header=0, index_col=0, sep="\t")
ngene, nsample = df.shape
gx_donors = list(df.columns)
gene_names = list(df.index)
gx = df.values
print(gx.shape)
print(np.linalg.matrix_rank(gx))


(13538, 605)
604


In [33]:
from iotools import readgtf

gtf_file = "/cbscratch/franco/datasets/GENCODE/gencode.v26.annotation.gtf.gz"
geneinfo = readgtf.gencode_v12(gtf_file, trim=False)

In [34]:
import copy
vcfmask, exprmask = select_donors(gt_donors, gx_donors)
genes, indices = select_genes(geneinfo, gene_names)       
expr = gx[:, exprmask][indices, :]
dosage_filtered_selected = gtfull[:, vcfmask]
geneinfo = genes

In [35]:
chrom=17
window=1e6
cismasklist = get_cismasklist(snp_info, geneinfo, chrom, window=window)
cismaskcomp = compress_cismasklist(cismasklist)

In [36]:
gt_cent, gt_norm = normalize_and_center_dosage(dosage_filtered_selected, snp_info)

gx_norm = normalize_expr(expr)
gx_corr, gt_corr = knn_correction(gx_norm.T, dosage_filtered_selected, 30)
gx_knn_norm = normalize_expr(gx_corr.T)
gt_knn_cent, gt_knn_norm = normalize_and_center_dosage(gt_corr, snp_info)

sigmax2 = np.var(gt_cent, axis = 1)

Original dimension: 605 x 13538
Reduced dimension: 605 x 605


In [37]:
SIGMA_BETAS = np.repeat( 0.1 ** 2, gt_knn_cent.shape[0])

In [54]:
def pvals_perm(GT, R, W):
    mu2, mu4 = moment_data(GT)
    N = GT.shape[1]
    q11 = np.sum(W)
    q2  = np.sum(np.diag(W))
    muQ = mu2 * (N * q2 - q11) / (N - 1)

    v31 = - mu4 / (N - 1)
    v22 = v31 + (N * mu2 * mu2 / (N - 1)) #(N*(mu2**2) - mu4)/(N-1)
    v211 = - (v31 + v22) / (N - 2)
    v1111 = - 3 * v211 / (N - 3)

    q31 = np.dot(np.diag(W),np.sum(W,axis = 1))
    q4 = np.sum(np.square(np.diag(W)))
    q22 = np.sum(np.square(W))
    q211 = np.sum(np.square(np.sum(W,axis = 1)))

    sigma2 = v1111*(q11**2 - 2*q2*q11 - 4*q211 + 8*q31 + 2*q22 + q2**2 - 6*q4) + 2*v211*(q2*q11 + 2*q211 - 6*q31 - 2*q22 - q2**2 + 6*q4) + v22*(q2**2 + 2*q22 - 3*q4) + 4*v31*(q31 - q4) + mu4*q4

    sigma2 = sigma2 - muQ**2
    sigmaQ = np.sqrt(sigma2)
    p = 1 - stats.norm.cdf(R, loc=muQ, scale=sigmaQ)
    return p, muQ, sigmaQ

def moment_data(GT):   #GT ixN
    GT2 = np.square(GT)
    GT4 = np.square(GT2)
    mu2 = np.mean(GT2)
    mu4 = np.mean(GT4)
    return mu2, mu4

def get_betas_force(gt, expr, sigma_x2, sb2):
    ngenes, nsamples = expr.shape
    nsnps = gt.shape[0]
    print("Samples:", nsamples)
    print("Genes:", ngenes)
    print("N SNPs:", nsnps)
    k = ngenes
    I = np.identity(k)
    
    B = np.zeros((nsnps, ngenes))
    YTY = np.matmul(expr, expr.T)
    for snpi in range(nsnps):
        Is  = I * sigma_x2[snpi]/sb2[snpi]
        W   = YTY + Is
        Winv= np.linalg.inv(W)
        A   = np.matmul(Winv, expr)
        Bi  = np.matmul(A, gt[snpi,:][np.newaxis].T)
        B[snpi,:] = Bi.reshape(-1)
    return B

def get_betas(gt, expr, sigma_x2, sb2):
    ngenes, nsamples = expr.shape
    nsnps = gt.shape[0]
    print("Samples:", nsamples)
    print("Genes:", ngenes)
    print("N SNPs:", nsnps)
    k = min(nsamples, ngenes)
    I = np.identity(k)
    
    U, s, VT = np.linalg.svd(expr.T, full_matrices=False)
    S = I*s  

    B = np.zeros((nsnps, ngenes))
    for snpi in range(nsnps):
        innerLinv_ST = (s / (s*s + (sigma_x2[snpi]/sb2[snpi]))) * I
        # print("innerLinv_ST:", innerLinv_ST.shape)
        UT = np.transpose(U)
        V = np.transpose(VT)
        # print("U:",U.shape)

        innerLinv_STUT = np.matmul(innerLinv_ST, UT)
        # print("innerLinv_STUT:", innerLinv_STUT.shape)  # if G < N, then UT is k x N and then innerL_STUT is k x N
        A2 = np.matmul(V, innerLinv_STUT)
        # print("A:", A2.shape)

        Bi = np.matmul(A2, gt[snpi,:][np.newaxis].T)
        B[snpi,:] = Bi.reshape(-1)
    return B

def reshape_masked_betas(b, mask, ngenes):
    print("reshaping {:d} betas into ({:d},{:d}) with {:d} masked genes out of {:d}".format(len(b), len(mask.apply2), (ngenes - len(mask.rmv_id)), len(mask.rmv_id), ngenes)) 
    _b = b.reshape(len(mask.apply2), ngenes-len(mask.rmv_id))
    paddedBeta = np.zeros( (len(mask.apply2), ngenes) )
    inv_ind = np.delete(np.arange(ngenes), mask.rmv_id)
    paddedBeta[:, inv_ind] = _b
    return paddedBeta.reshape(-1)

In [53]:
def compare_basic_props(GX, GT, sigmax2 = [0.1], sb2 = None, i = 0):
    # Yt = (GX / np.sqrt(nsample)).T
#     K = np.linalg.matrix_rank(GX)    
    nsnps  = GT.shape[0]
    ngenes = GX.shape[0]
    Rscore = np.zeros(nsnps)
    pvals  = np.zeros(nsnps)
    muQ    = np.zeros(nsnps)
    sigmaQ = np.zeros(nsnps)
    mu2    = np.zeros(nsnps)
    mu4    = np.zeros(nsnps)
    Keffs  = np.zeros(nsnps)
    for mask in cismaskcomp:
        applysnps = mask.apply2
        
        usegenes = np.ones(GX.shape[0], dtype=bool)
        if mask.rmv_id.shape[0] > 0: usegenes[mask.rmv_id] = False
        masked_gx = np.ascontiguousarray(GX[usegenes])
        
        print("Masked expression dims:", masked_gx.shape)
        Yt = masked_gx.T
        U, S, Vt = np.linalg.svd(Yt, full_matrices=False)
        S2 = np.square(S)
        
        for j in applysnps:       
            S2mod = S2 + sigmax2[j] / sb2[j]

            Keffs[j] = np.sum(S2 / S2mod)
            W = np.dot(U, np.dot(np.diag(S2 / S2mod), U.T)) / sigmax2[j]
            Rscore[j] = np.sum(np.square(np.dot(U.T, GT[j,:])) * (S2 / S2mod)) / sigmax2[j]

            pvals[j], muQ[j], sigmaQ[j] = pvals_perm(GT[j, :].reshape(1, -1), Rscore[j], W)
            mu2[j], mu4[j] = moment_data(GT[j, :].reshape(1, -1))
    
        betas_force = get_betas_force(GT, masked_gx, sigmax2, sb2)
        betas_force = reshape_masked_betas(betas_force, mask, ngenes)
        betas = get_betas(GT, masked_gx, sigmax2, sb2)
        betas = reshape_masked_betas(betas, mask, ngenes)
#     print("Qscore:", Rscore[:5])
#     print("muQ:", muQ[:5])
#     print("sigmaQ:", sigmaQ[:5])
#     print("pvals:", pvals[:5])
    return pvals, Rscore, muQ, sigmaQ, betas, betas_force

i = 0

print(gx_knn_norm.shape)
pvals, Qscores, muQ, sigmaQ, betas, betas_force = compare_basic_props(gx_knn_norm, gt_knn_cent, sigmax2 = sigmax2, sb2 = SIGMA_BETAS)


(13538, 605)
Masked expression dims: (13526, 605)
Samples: 605
Genes: 13526
N SNPs: 1
[[2.74844614 0.         0.         ... 0.         0.         0.        ]
 [0.         2.74844614 0.         ... 0.         0.         0.        ]
 [0.         0.         2.74844614 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 2.74844614 0.         0.        ]
 [0.         0.         0.         ... 0.         2.74844614 0.        ]
 [0.         0.         0.         ... 0.         0.         2.74844614]]
(13526, 13526)
reshaping 1 betas into (1,13526) with 12 masked genes out of 13538
Samples: 605
Genes: 13526
N SNPs: 1
reshaping 1 betas into (1,13526) with 12 masked genes out of 13538


In [55]:
betas

array([-3.94930681e-05, -2.01902428e-04,  2.74877836e-04, ...,
       -5.61347486e-05, -2.14336100e-05,  5.78134080e-04])

In [56]:
betas_old

array([-3.94930681e-05, -2.01902428e-04,  2.74877836e-04, ...,
       -5.61347486e-05, -2.14336100e-05,  5.78134080e-04])